In [20]:
import ecdsa
import hashlib



In [21]:
import random
accounts = { }
list_sk = []

for i in range(5):
    sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    list_sk.append(sk)
    pk = sk.verifying_key
    accounts[pk.to_string().hex()[:40]] = random.randint(0,100)

account_adress = list(accounts.keys())

In [22]:
class State :
    def __init__(self, accounts) :
        self.acc = accounts.keys()
        self.amo = accounts.values()
        self.accounts = accounts
    
    def printAccounts(self):
        print(self.accounts)
        
#Test
st1 = State(accounts)

st1.printAccounts()

{'81461273986fdfa81a19c94baaa9bc912047a0ba': 55, 'ff618f2c1f99707913ee73e11e41ff43be535b0b': 73, 'd7adc81d20cd57a89f842a3839c0be2c1159d3c5': 51, '67659975bb6cd2f6aed20dee32c056360bac3b40': 3, 'b0904ee3b8561663f4d929b84bc8b4cea505893c': 25}


In [23]:
class User:
    '''
    Containing serete key and public key as well as adress of the user
    Attributes:
     - secrete key
     - public key
     - adress
    '''
    def __init__(self, secrete_key, public_key, adress):
        self.sk = secrete_key
        self.pk = public_key
        self.adr = adress
    
class Transaction : 
    def __init__(self, sender, receiver, amount, signature = None): 
        '''
        Initial a transaction

        Arguments:
        - sender{user}: information of sender (sk and pk and adr)
        - receiver{hex}: adress of the receiver
        - amount{int}: the amount of coin sent
        - signature{bytes}: optinal parameter
        '''
        self.sender = sender        
        self.receiver = receiver    
        self.amount = amount        
        self.signature = signature  
    
    def buildSig(self, sk):
        '''
        Build a signature for a transaction 
        
        Arguments:
        - sk{ecdsa.keys.SigningKey}: secret key of the sender

        Return:
        - signature{bytes}: signature of the transaction
        '''
        #reciever + amount
        txData = str(self.receiver) + str(self.amount)

        #convert Data from string to bytes 
        txDataBytes = txData.encode()
        
        #the hash of Data
        self.hashOfData = hashlib.sha3_256(txDataBytes).digest() # output in bytes
        
        #create signature for this transaction
        self.signature = sk.sign(self.hashOfData) #in bytes 
          
        return self.signature
             
    def verifySig(self, signature):
        '''
        Verify the signature of the transaction

        Arguments:
        - signature{bytes}

        Return:
        -True: If the signature is correct
        -False: Otherwise
        '''
        try: 
            assert self.sender.pk.verify(signature, self.hashOfData)
            return True
        except:
            return False
            
    def recoverSenderAddressCandidates(self, signature):
        '''
        Recover the adress of the sender from its signature
        Arguments:
        - signature{bytes}: the signature of the transaction

        Return:
        - h[0],h[1] {hex}: List two potential values for the adress of the sender
        '''
        curve = ecdsa.curves.SECP256k1
        h = ecdsa.VerifyingKey.from_public_key_recovery(
		signature=signature, data=self.hashOfData, curve=curve, sigdecode=ecdsa.util.sigdecode_string)
        
        #return 2 potentials adress 
        return [h[0].to_string().hex()[0:40],h[1].to_string().hex()[0:40]]    
        
    def verifyTransaction(self, state): 
        '''
        Verify the transaction is satisfied by the conditions

        Arguments:
        - state{State}: the state of system

        Return:
        -True: If the transaction is correct
        -False: Otherwise
        '''
        #verify account exist in system
        h = []
        h = self.recoverSenderAddressCandidates(self.buildSig(self.sender.sk))
        if h[0] in state.acc:
            self.sender_account = h[0]
        elif h[1] in state.acc:
            self.sender_account = h[1]
        else:
            return False
        if self.receiver not in state.acc:
            print('receiver not in system')
            return False
        
        #verify signature is correct 
        
        if self.verifySig(self.buildSig(self.sender.sk)) == False:
            print('signature is not correct')
            return False
        
        #verify sender's account has enough money 
        
        if state.accounts[self.sender_account] < self.amount:
            print('sender does not have enough money')
            return False
     
        return True   
    
    def updateState(self, state):
        '''
        After a succes transaction, the state of system need to update

        Arguments:
        - state{State}: the state of system

        Return:
        - state{State}: new state of system

        '''
        if self.verifyTransaction(state) == True:
            state.accounts[self.sender_account] -= self.amount
            state.accounts[self.receiver] += self.amount
        

    def infors(self):
        '''
        Given info of the transaction

        Return:
        - sender{hex}: adress of the sender
        - receiver{hex}: adress of the receiver
        - amount{int}: the amount of coint sent
        '''
        return{
            'sender': self.sender.pk.to_string().hex()[:40],
            'receiver': self.receiver,
            'amount': self.amount
        }



In [24]:
#TEST
import pytest

# We already created a list 5 accounts include secrete key and public key
list_sk
# Now we create a profile for these 5 users
user = []
for i in range(5):
    user.append(User(list_sk[i], list_sk[i].verifying_key, list_sk[i].verifying_key.to_string().hex()[0:40]))

# Now we have the state of system
state = State(accounts)


def test_init():
    trans1 = Transaction(user[4], user[1].adr, 2)
    trans2 = Transaction(user[0], user[1].adr, 9)
    trans3 = Transaction(user[3], user[2].adr, None)
    trans4 = Transaction(user[1], user[2].adr, 1)

def test_buildSig():
    trans = Transaction(user[0], user[1].adr, 2)
    sign = trans.buildSig(user[0].sk)
    assert type(sign) == bytes
    assert len(sign) == 64

    print('Test buildSig--------->OK')
    
def test_verifySig():
    trans = Transaction(user[0], user[1].adr, 2)
    assert trans.verifySig(trans.buildSig(user[0].sk)) == True
    assert trans.verifySig(trans.buildSig(user[1].sk)) == False
    print('Test verifySig--------->OK')

def test_recoverSenderAdressCandidates():
    trans = Transaction(user[0], user[1].adr, 2)
    if user[0].adr in trans.recoverSenderAddressCandidates(trans.buildSig(user[0].sk)):
        print('Test recoverSenderAdressCandidates--------->OK')
    else:
        print('Test recoverSenderAdressCandidates--------->FALSE')
        
def test_veryfyTransaction():
    assert Transaction(user[0], user[1].adr, 2).verifyTransaction(state) == True
    assert Transaction(user[0], user[1].adr, 65).verifyTransaction(state) == False # the amount of user1 = 64
    # Create a stranger user
    sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    pk = sk.verifying_key
    stranger_user  = User(sk, pk, pk.to_string().hex()[0:40])
    assert Transaction(stranger_user, user[1], 9).verifyTransaction(state) == False # sender not in state
    assert Transaction(user[0],'ccekcmklekcma234mfkre4323md2', 9).verifyTransaction(state) == False # receiver not in state
    print('Test verifyTransaction--------->OK')

def test_updateState():
    trans = Transaction(user[0], user[1].adr, 2)
    const  = 0
    m0 = state.accounts[user[0].adr]
    m1 = state.accounts[user[1].adr]
    
    trans.updateState(state)

    assert m0 == state.accounts[user[0].adr] + 2
    assert m1 == state.accounts[user[1].adr] - 2
    print('Test updateState--------->OK')

    
    


if __name__ == '__main__':
    test_init()
    test_buildSig()
    test_verifySig()
    test_recoverSenderAdressCandidates()
    test_veryfyTransaction()
    test_updateState()
    trans = Transaction(user[0], user[1].adr, 2)


Test buildSig--------->OK
Test verifySig--------->OK
Test recoverSenderAdressCandidates--------->OK
sender does not have enough money
receiver not in system
Test verifyTransaction--------->OK
Test updateState--------->OK


In [25]:
class Block : 
    def __init__(self, index, previous_hash, transactions = None) : 
        '''
        Inital a block

        Arguments:
        -index{int}: index of the block
        -previous_hash{bytes}: the hash of prevous block
        -transaction{Transaction}: optional patrameter
        '''
        self.index = index
        self.previous_hash = previous_hash
        self.transactions = []
        self.current_hash = self.computeHash()
    def addTransaction(self, transaction):
        '''
        Add a transaction to the block

        Arguments:
        -transaction{Transaction}: new transaction want to add

        Return:
        -new state of block after adding new transaction (change in current_hash)

        '''
        self.transactions.append(transaction)
        self.current_hash = self.computeHash()
    def computeHash(self):
        '''
        Compute the hash of block

        Arguments:
        -None

        Return:
        -txHsah{bytes}: computed hashe of block
        '''
        
        tx = str(self.index) + str(self.transactions) + str(self.previous_hash)
        txBytes = tx.encode()
        txHash = hashlib.sha3_256(txBytes).digest()
        return txHash

    def verifyBlock(self, state):
        '''
        Verify the block is correct

        Arguments:
        -state{State}: the state of system

        Return:
        -True:if the block is correct
        -False: if the block is not correct
        '''
        #check if the hash of the block is correct
        if self.current_hash != self.computeHash():
            return False
        #check if all transactions in the block are valid
        for transaction in self.transactions:
            if not transaction.verifyTransaction(state):
                return False
        return True
    
    def infors(self):
        return{
            "Index": self.index,
            "Transactions": [t.infors() for t in self.transactions],
            "Previous Hash": self.previous_hash,
            "Hash": self.current_hash
        }
    @staticmethod
    def create_genesis_block():
        '''
        Create the genesis block
    
        '''
        return Block(0,'0', [])
    @staticmethod
    def next_block(last_block, transactions):
        '''
        Create the next block

        Arguments:
        -last_block{Block}: the last block of the chain
        -transactions{Transaction}: list of transactions in the new block
        '''
        index = last_block.index +1
        previous_hash = last_block.current_hash
        return Block(index, previous_hash, transactions)




In [26]:
#TEST
trans1 = Transaction(user[4], user[1].adr, 2)
trans2 = Transaction(user[0], user[1].adr, 9)
trans3 = Transaction(user[3], user[2].adr, None)
trans4 = Transaction(user[1], user[2].adr, 1)

def test_init():
    gens_block = Block.create_genesis_block()
    block1 = Block.next_block(gens_block, [])
    block2 = Block(9,block1.computeHash() )
    print('test_init -------------->OK')

def test_addTransaction():
    gens_block = Block.create_genesis_block()
    block1 = Block.next_block(gens_block, [])
    block2 = Block(9,block1.computeHash() )
    block2.addTransaction(trans1)
    block2.addTransaction(trans2)
    assert block2.transactions[0] == trans1
    assert block2.transactions[1] == trans2
    print('Test addTransaction -------------->OK')

def test_computedHash():
    gens_block = Block.create_genesis_block()
    block1 = Block.next_block(gens_block, [])
    block2 = Block(9,block1.computeHash() )
    assert type(block2.computeHash()) == bytes
    print('Test computeHash -------------->OK')

def test_verifyBlock():
    gens_block = Block.create_genesis_block()
    block1 = Block.next_block(gens_block, [])

    block2 = Block(9,block1.computeHash() )
    block2.addTransaction(trans1)
    block2.addTransaction(trans2)
    assert block2.verifyBlock(state) == True
    block2.addTransaction(Transaction(user[0],'ccekcmklekcma234mfkre4323md2', 9)) # add a false transaction
    assert block2.verifyBlock(state) == False

    block3 = Block(4,'cnwoje9902ncwccww09jew000887')
    assert block3.verifyBlock(state) == True
    print('Test verifyBlock -------------->OK')


    


if __name__ == '__main__':

    test_init()
    test_addTransaction()
    test_computedHash()
    test_verifyBlock()


test_init -------------->OK
Test addTransaction -------------->OK
Test computeHash -------------->OK
receiver not in system
Test verifyBlock -------------->OK


In [27]:
class Blockchain : 
    def __init__(self, state):
        self.state = state
        self.blocks = []
    def addBlock(self, newblock):
        '''
        Add a new block for the chain

        Arguments:
        -newblock{Block}: the block want to add

        Return:
        -new blockchain
        '''
        if len(self.blocks) == 0:
            self.blocks.append(Block.create_genesis_block())
        if newblock.verifyBlock(state) == True: 
            
            if newblock.previous_hash == self.blocks[-1].current_hash:
                self.blocks.append(newblock)
                for transaction in newblock.transactions:
                    transaction.updateState(self.state)
                return True
            else:
                print('The new block is not connected to the chain - blocks index: ', newblock.index)
                return False
        
        else:
            print('Invalid block ! - blocks index: ', newblock.index)
            return False
        

In [28]:
#TEST
trans1 = Transaction(user[4], user[1].adr, 1)
trans2 = Transaction(user[0], user[1].adr, 1)
trans3 = Transaction(user[3], user[2].adr, None)
trans4 = Transaction(user[1], user[2].adr, 1)
False_transaction = Transaction(user[0],'ccekcmklekcma234mfkre4323md2', 9)  # transation not exist in system
state = State(accounts)

def test_addBlock():

    blockchain = Blockchain(state)  # initial a block chain

    gens_block = Block.create_genesis_block()
    block1 = Block(1, gens_block.computeHash()) # create the first block index 1
    assert blockchain.addBlock(block1) == True       # add the first block to the vide blockchain        

    block2 = Block(2,block1.current_hash)
    block2.addTransaction(trans1)      #create the second block
    assert blockchain.addBlock(block2) == True       # add the seocond block to the blockchain

    block3 = Block(3, block1.current_hash) #create the thrid block
    block3.addTransaction(trans2)
    assert blockchain.addBlock(block3) == False   # because it not connects to the last block (block 2)

    block4 = Block(4, block2.current_hash)       # create the fouth transaction
    block4.addTransaction(False_transaction)
    assert blockchain.addBlock(block4) == False # because it containt a transaction which does not exist in system

    print('Test addBlock----------------->OK')


if __name__=='__main__':
    state.printAccounts() # state before add block
    test_addBlock()
    state.printAccounts() #state after addd block

{'81461273986fdfa81a19c94baaa9bc912047a0ba': 53, 'ff618f2c1f99707913ee73e11e41ff43be535b0b': 75, 'd7adc81d20cd57a89f842a3839c0be2c1159d3c5': 51, '67659975bb6cd2f6aed20dee32c056360bac3b40': 3, 'b0904ee3b8561663f4d929b84bc8b4cea505893c': 25}
The new block is not connected to the chain - blocks index:  3
receiver not in system
Invalid block ! - blocks index:  4
Test addBlock----------------->OK
{'81461273986fdfa81a19c94baaa9bc912047a0ba': 53, 'ff618f2c1f99707913ee73e11e41ff43be535b0b': 76, 'd7adc81d20cd57a89f842a3839c0be2c1159d3c5': 51, '67659975bb6cd2f6aed20dee32c056360bac3b40': 3, 'b0904ee3b8561663f4d929b84bc8b4cea505893c': 24}
